In [1]:

%pprint
import sys

# Insert at 1, 0 is the script path (or '' in REPL)
if ('../py' not in sys.path): sys.path.insert(1, '../py')
from jobpostlib import (cu, datetime, duration, freq, hau, humanize, nu, scrfcu, slrcu, su, t0, time, wsu, speech_engine)
import os
from urllib.parse import urlparse, parse_qs
import re
import warnings
from urllib.parse import urlparse, parse_qs, urlencode
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
from bs4.formatter import  HTMLFormatter

warnings.filterwarnings('ignore')
job_title_css = 'h1.t-24'
job_subtitle_css = 'div.jobs-unified-top-card__primary-description > span.jobs-unified-top-card__subtitle-primary-grouping'
ascii_regex = re.compile('[^A-Za-z0-9]+')
article_css = '.jobs-description__container'
seemore_button_css = 'button.t-14'
seemore_button_xpath = '/html/body//footer/button'
details_css = '#job-details'
nav_search_css = '.global-nav__content'

Pretty printing has been turned OFF
======== Neo4j/4.4.7 ========
Utility libraries created in 14 seconds



----

In [2]:

wsu.save_linkedin_email_to_file(verbose=True)

['powershell.exe', '-ExecutionPolicy', 'Bypass', '-Command', 'Start-Process "C:\\Program Files\\Notepad++\\notepad++.exe" -ArgumentList "C:\\Users\\daveb\\OneDrive\\Documents\\GitHub\\job-hunting\\data\\html\\linkedin_email.html"']


In [3]:

try:
    driver = wsu.get_driver(verbose=True)
    wsu.log_into_linkedin(driver)
    speech_engine.say('Enter the verification code'); speech_engine.runAndWait()
finally:
    page_soup = wsu.get_page_soup('../data/html/linkedin_email.html')
    css_selector = 'a[href^="https://www.linkedin.com/comm/jobs/view"]'
    link_soups_list = page_soup.select(css_selector)
    
    # Get rid of the duplicate URLs
    url_strs_set = set()
    for link_soup in link_soups_list:
        
        # URL string
        url_str = link_soup['href']
        
        # Parse the URL
        parsed_url = urlparse(url_str)
        
        # Get query parameters as dictionary
        query_params = parse_qs(parsed_url.query)
        
        # Remove trk and trkEmail parameters if they exist
        if 'trk' in query_params:
            del query_params['trk']
        if 'trkEmail' in query_params:
            del query_params['trkEmail']
        
        # Construct the updated query string
        updated_query_str = urlencode(query_params, doseq=True)
        
        # Reconstruct the URL with the updated query string
        updated_url = parsed_url.scheme + '://' + parsed_url.netloc + parsed_url.path
        if updated_query_str:
            updated_url += '?' + updated_query_str
        
        url_strs_set.add(updated_url)
    
    display(len(url_strs_set))

Getting the FireFox driver
platform.system() = Windows
os.name = nt
gecko_driver_path = GeckoDriverManager().install() = C:\Users\daveb\.wdm\drivers\geckodriver\win64\v0.34.0\geckodriver.exe
Getting URL: https://www.linkedin.com/home
Clicking /html/body/nav/div/a[2]
Filling in the session_key field with dave.babbitt@gmail.com
Clicking /html/body/div/main/div[2]/div[1]/form/div[3]/button


2

In [4]:

# Assumes this is the first time you've run this cell
files_list = []

for url_str in url_strs_set:
    wsu.driver_get_url(driver, url_str, verbose=False)
    time.sleep(4)
    titles_list = driver.find_elements(By.CSS_SELECTOR, 'h1.t-24')
    if not titles_list:
        titles_list = driver.find_elements(By.CSS_SELECTOR, '#ember130 > h2:nth-child(1)')
    if titles_list:
        job_title_str = titles_list[0].text
        job_subtitle_str = driver.find_elements(By.CSS_SELECTOR, 'span.tvm__text:nth-child(1)')[0].text
        page_title = f'{job_title_str} {job_subtitle_str}'
        # print(page_title)
        file_name = ascii_regex.sub(' ', page_title).strip().replace(' ', '_')
        trackingId = ascii_regex.sub(' ', parse_qs(urlparse(url_str).query).get('trackingId', [''])[0]).strip().replace(' ', '_')
        if len(trackingId):
            file_name = f'{trackingId}_{file_name}.html'
        else:
            file_name = f'{file_name}.html'

            # Assumes this is the first time you've run this cell
            file_path = os.path.join(cu.SAVES_HTML_FOLDER, file_name)
            if os.path.isfile(file_path):
                file_name = datetime.now().strftime('%Y%m%d%H%M%S%f') + f'_{file_name}'

        file_path = os.path.join(cu.SAVES_HTML_FOLDER, file_name)
        if not os.path.isfile(file_path):
            
            # Save the HTML to the file
            with open(file_path, 'w', encoding=nu.encoding_type) as f:
                print(f'Saving to {file_path}')
                f.write('<html>\n    <head>\n        <title>')
                f.write(page_title)
                f.write(
                    '</title>\n    </head>\n    <body>\n        <div class="jobsearch-JobComponent-description">'
                    + '\n            <div class="jobsearch-JobComponent-description" id="jobDescriptionText">\n'
                )
                overlay_tag = driver.find_elements(By.CSS_SELECTOR, nav_search_css)[0]
                driver.execute_script("arguments[0].setAttribute('style','display:none;');", overlay_tag)
                wsu.click_by_xpath(driver, seemore_button_xpath, verbose=False)
                web_obj = driver.find_elements(By.CSS_SELECTOR, details_css)[0]
                article_str = web_obj.get_attribute('innerHTML').strip()
                
                # Prettify the HTML
                formatter_obj = HTMLFormatter(indent=4)
                page_soup = bs(article_str, 'html.parser')
                html_str = page_soup.prettify(formatter=formatter_obj)
                
                f.write(re.sub('^', '                ', html_str, 0, re.MULTILINE))
                f.write('</div></body></html>')
            
            # Delete the svg tags and tighten up the parent tag for easier viewing
            with open(file_path, 'r', encoding=nu.encoding_type) as f:
                html_str = f.read()
            html_str = re.sub('<svg[^>]*>(<path[^>]*></path>)+</svg>', '', html_str)
            html_str = re.sub('\s*<style[^>]*>[^><]*</style>', '', html_str)
            html_str = re.sub(r'<([^></ ]+)([^></]*)>[\r\n]+ +([^><\r\n]+)[\r\n]+ +</\1>', r'<\1\2>\3</\1>', html_str)
            with open(file_path, 'w', encoding=nu.encoding_type) as f:
                f.write(html_str)
            
            files_list.append(file_name)
        cu.ensure_filename(file_name, verbose=False)
        cu.set_posting_url(file_name, url_str, verbose=False)
speech_str = f'Fileing {len(files_list)} postings complete. Delete the email.'; print(speech_str); speech_engine.say(speech_str); speech_engine.runAndWait()

Saving to C:\Users\daveb\OneDrive\Documents\GitHub\job-hunting\saves\html\FS8seU7HWR9zf11ye36rlQ_Machine_Learning_Operations_Engineer_II_PySpark_Python_R_Milford_MA.html
Saving to C:\Users\daveb\OneDrive\Documents\GitHub\job-hunting\saves\html\Mc3nBYaQbit9hBuZUC7R7g_Machine_Learning_Engineer_Massachusetts_United_States.html
Fileing 2 postings complete. Delete the email.


In [5]:

t1 = time.time()
try:
    driver.close()
except Exception as e:
    print(f'{e.__class__.__name__} error: {str(e).strip()}')
cu.ensure_navigableparent('END', verbose=False)
for file_name in files_list:
    file_path = os.path.join(cu.SAVES_HTML_FOLDER, file_name)
    page_soup = wsu.get_page_soup(file_path)
    row_div_list = page_soup.find_all(name='article')
    for div_soup in row_div_list:
        child_strs_list = hau.get_navigable_children(div_soup, [])
        assert child_strs_list, f'Something is wrong with {file_name}'
        cu.populate_from_child_strings(child_strs_list, file_name, verbose=False)
duration_str = humanize.precisedelta(time.time() - t1, minimum_unit='seconds', format='%0.0f')
speech_str = f'Populating {len(files_list)} out of {len(url_strs_set)} postings completed in {duration_str}'; speech_engine.say(speech_str); speech_engine.runAndWait()
files_list

['FS8seU7HWR9zf11ye36rlQ_Machine_Learning_Operations_Engineer_II_PySpark_Python_R_Milford_MA.html', 'Mc3nBYaQbit9hBuZUC7R7g_Machine_Learning_Engineer_Massachusetts_United_States.html']